# Modules

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime, date, time
import seaborn as sns
from scipy import stats
import scipy.stats as st
import glob

# Dataset

The dataset used in this analysis consists of beer reviews from two beer rating websites,**BeerAdvocate** and **RateBeer**, for a period ranging from 2001 to 2017. For each website, we have 5 files:
- users.csv: metadata about reviewers
- beers.csv : metadata about reviewed beers
- breweries.csv : metadata about breweries
- ratings.txt : all reviews given by users, including numerical ratings and sometimes textual reviews
- reviews.txt : only reviews given by users that include both numerical ratings and textual reviews

In our analysis, we will not use textual reviews. Thus, we will only use ratings.txt files and not reviews.txt files, as we will use all reviews, whether or not they include textual reviews.

### Load data into Dataframes

The .csv files are not too large and can efficiently be loaded into DataFrames.

In [2]:
BA_DATA_FOLDER = 'data/BeerAdvocate/'
RB_DATA_FOLDER = 'data/RateBeer/'

BA_USERS = BA_DATA_FOLDER+"users.csv"
BA_BEERS = BA_DATA_FOLDER+"beers.csv"
BA_BREWERIES = BA_DATA_FOLDER+"breweries.csv"

RB_USERS = RB_DATA_FOLDER+"users.csv"
RB_BEERS = RB_DATA_FOLDER+"beers.csv"
RB_BREWERIES = RB_DATA_FOLDER+"breweries.csv"

In [3]:
ba_users = pd.read_csv(BA_USERS)
ba_beers = pd.read_csv(BA_BEERS)
ba_breweries = pd.read_csv(BA_BREWERIES)

rb_users = pd.read_csv(RB_USERS)
rb_beers = pd.read_csv(RB_BEERS)
rb_breweries = pd.read_csv(RB_BREWERIES)

On the other hand, the ratings.txt files are extremely large, and trying to load them directly into DataFrames leads to kernel freezes. In order to circumvent this problem, we wrote a script (review_parser.py, located in src/scripts), which processes each rating file by dividing it into parts, parsing each part, and saving as JSON. In the notebook, we then load the different JSON files into DataFrames, that we concatenate. Dividing the large .txt files into smaller JSON chunks and then loading each chunk separately, avoids trying to load the entire file into memory at once, which can cause kernel freezes due to memory overload. In addition, JSON is a format that pandas can read efficiently.

In [4]:
# Load BeerAdvocate ratings stored in json files into a single DataFrame
ba_json_files = glob.glob(BA_DATA_FOLDER+'*.json')
ba_df_list = [pd.read_json(file) for file in ba_json_files]
ba_ratings = pd.concat(ba_df_list, ignore_index=True)
ba_ratings.head()

C:\Users\orkun\AppData\Local\Temp\ipykernel_31000\3299018877.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  ba_df_list = [pd.read_json(file) for file in ba_json_files]
C:\Users\orkun\AppData\Local\Temp\ipykernel_31000\3299018877.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  ba_df_list = [pd.read_json(file) for file in ba_json_files]


,beer_name,beer_id,brewery_name,brewery_id,style,abv,date,user_name,user_id,appearance,aroma,palate,taste,overall,rating
0,Régab,142544.0,Societe des Brasseries du Gabon (SOBRAGA),37262.0,Euro Pale Lager,4.5,2015-08-20 09:59:28,nmann08,nmann08.184925,3.25,2.75,3.25,2.75,3.0,2.88
1,Barelegs Brew,19590.0,Strangford Lough Brewing Company Ltd,10093.0,English Pale Ale,4.5,2009-02-20 10:59:12,StJamesGate,stjamesgate.163714,3.00,3.50,3.50,4.00,3.5,3.67
2,Barelegs Brew,19590.0,Strangford Lough Brewing Company Ltd,10093.0,English Pale Ale,4.5,2006-03-13 10:59:12,mdagnew,mdagnew.19527,4.00,3.50,3.50,4.00,3.5,3.73
3,Barelegs Brew,19590.0,Strangford Lough Brewing Company Ltd,10093.0,English Pale Ale,4.5,2004-12-01 10:59:12,helloloser12345,helloloser12345.10867,4.00,3.50,4.00,4.00,4.5,3.98
4,Barelegs Brew,19590.0,Strangford Lough Brewing Company Ltd,10093.0,English Pale Ale,4.5,2004-08-30 09:59:28,cypressbob,cypressbob.3708,4.00,4.00,4.00,4.00,4.0,4.00


In [5]:
# Load RateBeer ratings stored in json files into a single DataFrame
rb_json_files = glob.glob(RB_DATA_FOLDER+'*.json')
rb_df_list = [pd.read_json(file) for file in rb_json_files]
rb_ratings = pd.concat(rb_df_list, ignore_index=True)
rb_ratings.head()

C:\Users\orkun\AppData\Local\Temp\ipykernel_31000\180671459.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  rb_df_list = [pd.read_json(file) for file in rb_json_files]
C:\Users\orkun\AppData\Local\Temp\ipykernel_31000\180671459.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  rb_df_list = [pd.read_json(file) for file in rb_json_files]


,beer_name,beer_id,brewery_name,brewery_id,style,abv,date,user_name,user_id,appearance,aroma,palate,taste,overall,rating
0,33 Export (Gabon),410549.0,Sobraga,3198.0,Pale Lager,5.0,2016-04-26 10:00:00,Manslow,175852.0,2.0,4.0,2.0,4.0,8.0,2.0
1,Castel Beer (Gabon),105273.0,Sobraga,3198.0,Pale Lager,5.2,2017-02-17 11:00:00,MAGICuenca91,442761.0,2.0,3.0,2.0,4.0,8.0,1.9
2,Castel Beer (Gabon),105273.0,Sobraga,3198.0,Pale Lager,5.2,2016-06-24 10:00:00,Sibarh,288889.0,3.0,3.0,2.0,3.0,5.0,1.6
3,Castel Beer (Gabon),105273.0,Sobraga,3198.0,Pale Lager,5.2,2016-01-01 11:00:00,fombe89,250510.0,4.0,3.0,1.0,2.0,5.0,1.5
4,Castel Beer (Gabon),105273.0,Sobraga,3198.0,Pale Lager,5.2,2015-10-23 10:00:00,kevnic2008,122778.0,2.0,4.0,2.0,4.0,7.0,1.9


### First look at the data

We will now examine the different DataFrames in more detail.

In [6]:
# explain the columns of users, beers, breweries and ratings DataFrames

**BeerAdvocate beer Dataframe**

In [7]:
ba_beers.sample(4)

,beer_id,beer_name,brewery_id,brewery_name,style,nbr_ratings,nbr_reviews,avg,ba_score,bros_score,abv,avg_computed,zscore,nbr_matched_valid_ratings,avg_matched_valid_ratings
243780,276875,Progressive #1,42212,Cold Creek Brewery,American IPA,4,1,4.55,NaN,NaN,8.2,4.235000,0.515709,1,4.66
269801,13053,Breakfast Buzz,4871,Deep Creek Brewing Company,American Stout,0,0,NaN,NaN,NaN,4.2,NaN,NaN,0,NaN
886,94810,Innis & Gunn Spiced Rum Aged,10272,Innis & Gunn,Scottish Ale,33,4,3.95,86.0,NaN,7.4,4.025758,NaN,0,NaN
72012,134793,Velvet Green,683,Valkyrie Brewing,Irish Dry Stout,3,1,3.24,NaN,NaN,4.6,3.103333,NaN,0,NaN


Let us explain the different columns of the BeerAdvocate beer Dataframe, in which each row is a beer:
- beer_id, beer_name, brewery_id, brewery_name, style are explicit
- nbr_ratings: total number of reviews for that beer, whether or not they include textual reviews
- nbr_reviews: number of reviews for that beer that include textual reviews
- avg: average rating (out of 5) given to the beer based on user ratings
- ba_score: the BeerAdvocate score assigned to the beer, which corresponds to the beer's overall rating within its style category, calculated using a trimmed mean and a custom Bayesian formula that adjusts for the beer's style, balancing the score based on the number of ratings and the style's average
- bros_score: beer rating given by the site’s founders
- abv: 'Alcohol by volume', which indicates the percentage of alcohol content in the beer
- avg_computed: average rating (out of 5) recalculated using a weighted sum of the different aspect ratings
- zscore: z-score of the beer's average rating, which is a statistical measure that indicates how many standard deviations the average rating is from the mean of all ratings from the BeerAdvocate dataset
- nbr_matched_valid_ratings: number of valid ratings for beers that were successfully matched between two BeerAdvocate and RateBeer
- avg_matched_valid_ratings: average rating of those matched and valid ratings across the sites

The last two columns are related to the analysis performed by Robert West and Gael Lederrey in the following paper: https://dlab.epfl.ch/people/west/pub/Lederrey-West_WWW-18.pdf.

**RateBeer beer Dataframe**

In [8]:
rb_beers.sample(4)

,beer_id,beer_name,brewery_id,brewery_name,style,nbr_ratings,overall_score,style_score,avg,abv,avg_computed,zscore,nbr_matched_valid_ratings,avg_matched_valid_ratings
254498,445460,Flix Carmel Tripel Carmel-ite,24598,Flix Brewhouse Carmel,Abbey Tripel,1,NaN,NaN,3.11,9.7,3.80,NaN,0,NaN
340483,100466,Boulder Dam Reclamation Amber,8862,Boulder Dam Brewing Co.,Amber Ale,5,NaN,NaN,2.68,NaN,2.60,NaN,0,NaN
95806,338657,Aribayos Mesopotamia Imperial Stout,19281,Abadía de Aribayos,Imperial Stout,10,79.0,21.0,3.41,8.0,3.59,NaN,0,NaN
384109,448556,Pictish Ruadh,1950,Pictish,Amber Ale,2,NaN,NaN,3.05,4.3,3.30,NaN,0,NaN


Let us explain the different columns of the RateBeer beer Dataframe, in which each row is a beer:

The beer_id, beer_name, brewery_id, brewery_name, style, nbr_ratings, avg, abv, avg_computed, z-score, nbr_matched_valid_ratings and avg_matched_valid_ratings are the same as for the BeerAdvocate beer Dataframe.

Some columns are missing compared to the BeerAdvocate beer Dataframe: ba_score and bros_score (which makes sense as these are BeerAdvocate-specific scores), and nbr_reviews.

New columns are present compared to the BeerAdvocate beer Dataframe:
- overall_score: score (out of 100) which "reflects the rating given by RateBeer users and how this beer compares to all other beers on RateBeer", calculated by considering the ratings given by each user and the total number of ratings for the beer
- style_score: score given to the beer (out of 100) specifically within its style category

**BeerAdvocate user Dataframe**

In [9]:
ba_users.sample(4)

,nbr_ratings,nbr_reviews,user_id,user_name,joined,location
9426,1,0,gigot.881592,Gigot,1.413886e+09,NaN
924,50,50,gdogalishus.9254,gdogalishus,1.097316e+09,"United States, Pennsylvania"
69357,3,0,kittysaurus_rex.879007,Kittysaurus_Rex,1.413454e+09,"United States, Massachusetts"
34233,21,20,stoutmasterjames.435779,stoutmasterjames,1.268046e+09,"United States, Maryland"


Let us explain the different columns of the BeerAdvocate user Dataframe, in which each row is a reviewer:
- nbr_ratings, nbr_reviews, user_id, user_name, and location are explicit
- joined: timestamp indicating when the user joined BeerAdvocate in Unix timestamp format (the number of seconds since January 1, 1970, 00:00:00 UTC)

**RateBeer user Dataframe**

In [10]:
rb_users.sample(4)

,nbr_ratings,user_id,user_name,joined,location
69492,1,50990,GeertGouwy,1.173265e+09,Belgium
21880,52,280765,spatenfan,1.380017e+09,"United States, Wisconsin"
47579,2,284276,jillk1,1.381918e+09,NaN
53947,2,310491,stimutacs,1.396174e+09,NaN


Let us explain the different columns of the RateBeer user Dataframe, in which each row is a reviewer:

The columns are the same as in the BeerAdvocate user Dataframe (joined is obviously the timestamp indicating when the user joined RateBeer and not BeerAdvocate), except that nbr_reviews is missing.

**Brewery Dataframes**

In [11]:
ba_breweries.sample(4)

,id,location,name,nbr_beers
996,8669,England,Edale Brewery Company Limited,0
3817,5882,Germany,Bräustatt und Taverne Simmerberg,0
2674,9781,Japan,Taisetsu Ji Beer,7
9315,46857,"United States, Washington",Gruff Brewing,15


In [12]:
rb_breweries.sample(4)

,id,location,name,nbr_beers
4203,12071,Germany,Wasgau Wentzler-Bräu,9
2518,5216,Bulgaria,Pivovaren Zavod Varnensko Pivo (Ledenika i MM ...,2
2367,3691,Hungary,Blonder Sörgyar,2
2080,11521,Japan,Chitei Ryoko,8


The columns are explicit and are the same for the 2 websites. Each row is a brewery.

**Rating Dataframes**

In [13]:
ba_ratings.sample(4)

,beer_name,beer_id,brewery_name,brewery_id,style,abv,date,user_name,user_id,appearance,aroma,palate,taste,overall,rating
4358462,Green Tea Imperial Stout (Brewhouse Rarities),91320.0,Flying Dog Brewery,68.0,American Double / Imperial Stout,10.0,2013-10-23 10:00:00,maxk,maxk.264119,NaN,NaN,NaN,NaN,NaN,4.00
5472775,Hop Dish IPA,69316.0,Lift Bridge Brewery,18563.0,American IPA,7.5,2016-01-23 11:00:00,Tyecko,tyecko.789398,4.00,4.0,4.00,4.0,4.0,4.00
5493067,Parapet ESB,130250.0,Northgate Brewing,30076.0,Extra Special / Strong Bitter (ESB),5.6,2015-04-25 10:00:00,BigAl18,bigal18.870417,3.75,3.5,3.25,3.5,3.5,3.49
3502340,Evil Twin / Intangible Ales Sour Bikini,82842.0,Evil Twin Brewing,24300.0,American Wild Ale,3.0,2015-12-29 11:00:00,VikeMan,vikeman.348498,4.00,4.0,4.00,4.0,4.0,4.00


In [14]:
rb_ratings.sample(4)

,beer_name,beer_id,brewery_name,brewery_id,style,abv,date,user_name,user_id,appearance,aroma,palate,taste,overall,rating
1087926,Hofbrau Freising Dunkel,14915.0,Hofbrauhaus Freising &#40;Toerring&#41;,1802.0,Dunkel/Tmavý,4.8,2012-09-16 10:00:00,kevnic2008,122778.0,3.0,5.0,3.0,5.0,12.0,2.8
3639414,Butternuts Snapperhead IPA,69310.0,Butternuts Beer and Ale,6863.0,India Pale Ale (IPA),6.8,2009-08-05 09:59:28,williamherbert,93481.0,4.0,6.0,3.0,5.0,14.0,3.2
132107,Dieu du Ciel Péché Mortel,11461.0,Dieu du Ciel,364.0,Imperial Stout,9.5,2005-08-03 10:00:00,DrunkAsASkunk,5212.0,5.0,9.0,4.0,9.0,17.0,4.4
6041072,Great Heck / Steel City Yule Twig,238405.0,Great Heck,10061.0,Premium Bitter/ESB,5.2,2013-11-17 10:59:12,maeib,10280.0,3.0,7.0,3.0,7.0,14.0,3.4


The columns are the same for the 2 Dataframes. Each row corresponds to an individual review. Most column names are explicit. 
- 'appearance','aroma', 'palate','taste' correspond to aspect ratings (out of 5)
- 'overall' is the mean of the 4 aspect ratings
- 'rating' is the final rating given by the user to the beer

# 0) Data cleaning

In [15]:
# remove useless columns (done)
# make sure each column has the right type (done)
# deal with missing or Nan values (done)
# check the correspondance between brewery_id in the beers DataFrames and brewery_id in the breweries Dataframes (done)
# set all US locations to 'United States' (remove state information) (done)
# remove any embedded HTML links in the location strings (done)
# remove countries with too few reviewers (done)

## Filtering Dataframes

Let us start by removing columns in the different Dataframes that we will not use in our analysis.

The following rows will not be used in our analysis:
nbr_reviews, ba_score, bros_score, abv, avg_computed, zscore, nbr_matched_valid_ratings and avg_matched_valid_ratings, overall_score and style_score.

Let us remove them.

In [16]:
useless_columns_ba = ['nbr_reviews', 'ba_score', 'bros_score', 'abv', 'avg_computed', 'zscore', 'nbr_matched_valid_ratings', 'avg_matched_valid_ratings']
ba_beers = ba_beers.drop(columns=useless_columns_ba)
print(ba_beers.columns)

Index(['beer_id', 'beer_name', 'brewery_id', 'brewery_name', 'style',
       'nbr_ratings', 'avg'],
      dtype='object')


In [17]:
useless_columns_rb = [col for col in useless_columns_ba if col not in ['nbr_reviews','ba_score', 'bros_score']] + ['overall_score', 'style_score']
rb_beers = rb_beers.drop(columns=useless_columns_rb)
print(rb_beers.columns)

Index(['beer_id', 'beer_name', 'brewery_id', 'brewery_name', 'style',
       'nbr_ratings', 'avg'],
      dtype='object')


We will also not use the timestamps indicating the time when users joined the platforms, so let us remove this as well.

In [18]:
ba_users = ba_users.drop(columns='joined')
rb_users = rb_users.drop(columns='joined')
print(ba_users.columns)

Index(['nbr_ratings', 'nbr_reviews', 'user_id', 'user_name', 'location'], dtype='object')


## Verifying value types

Let us verify that the values in the different columns of the different Dataframes have the appropriate type.

In [19]:
print(ba_beers.dtypes,'\n','\n',rb_beers.dtypes)

beer_id           int64
beer_name        object
brewery_id        int64
brewery_name     object
style            object
nbr_ratings       int64
avg             float64
dtype: object 
 
 beer_id           int64
beer_name        object
brewery_id        int64
brewery_name     object
style            object
nbr_ratings       int64
avg             float64
dtype: object


In [20]:
print(ba_users.dtypes,'\n','\n',rb_users.dtypes)

nbr_ratings     int64
nbr_reviews     int64
user_id        object
user_name      object
location       object
dtype: object 
 
 nbr_ratings     int64
user_id         int64
user_name      object
location       object
dtype: object


In [21]:
print(ba_breweries.dtypes,'\n','\n',rb_breweries.dtypes)

id            int64
location     object
name         object
nbr_beers     int64
dtype: object 
 
 id            int64
location     object
name         object
nbr_beers     int64
dtype: object


In [22]:
columns_to_convert = ['beer_name', 'brewery_name', 'style']

ba_beers[columns_to_convert] = ba_beers[columns_to_convert].apply(lambda col: col.astype(str))
rb_beers[columns_to_convert] = rb_beers[columns_to_convert].apply(lambda col: col.astype(str))
print(ba_beers.dtypes,'\n','\n',rb_beers.dtypes)

beer_id           int64
beer_name        object
brewery_id        int64
brewery_name     object
style            object
nbr_ratings       int64
avg             float64
dtype: object 
 
 beer_id           int64
beer_name        object
brewery_id        int64
brewery_name     object
style            object
nbr_ratings       int64
avg             float64
dtype: object


In [23]:
print(ba_ratings.dtypes,'\n','\n',rb_ratings.dtypes)

beer_name               object
beer_id                float64
brewery_name            object
brewery_id             float64
style                   object
abv                    float64
date            datetime64[ns]
user_name               object
user_id                 object
appearance             float64
aroma                  float64
palate                 float64
taste                  float64
overall                float64
rating                 float64
dtype: object 
 
 beer_name               object
beer_id                float64
brewery_name            object
brewery_id             float64
style                   object
abv                    float64
date            datetime64[ns]
user_name               object
user_id                float64
appearance             float64
aroma                  float64
palate                 float64
taste                  float64
overall                float64
rating                 float64
dtype: object


The types of the values in the different columns of the different Dataframes seem appropriate.

## Dealing with missing values

In [24]:
ba_beers['avg'].value_counts()

avg
4.00    7783
3.75    7059
3.50    5946
3.88    3307
4.25    2871
        ... 
1.14       1
1.19       1
1.04       1
1.05       1
1.27       1
Name: count, Length: 401, dtype: int64

In [25]:
ba_beers

,beer_id,beer_name,brewery_id,brewery_name,style,nbr_ratings,avg
0,166064,Nashe Moskovskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,NaN
1,166065,Nashe Pivovskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,NaN
2,166066,Nashe Shakhterskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,NaN
3,166067,Nashe Zhigulevskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,NaN
4,166063,Zhivoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,NaN
...,...,...,...,...,...,...,...
280818,19139,Kölsch Ale,885,Summit Station Restaurant & Brewery,Kölsch,3,2.71
280819,19140,Nut Brown Ale,885,Summit Station Restaurant & Brewery,English Brown Ale,2,3.10
280820,19146,Octoberfest,885,Summit Station Restaurant & Brewery,Märzen / Oktoberfest,0,NaN
280821,2805,Scotch Ale,885,Summit Station Restaurant & Brewery,Scotch Ale / Wee Heavy,0,NaN


In [26]:
ba_beers_cleaned = ba_beers[~pd.isna(ba_beers['avg'])].reset_index() # avg = NaN valued beers are removed since there are not any reviews
ba_beers_cleaned

"""
# We are checking if there are beers with 0 averages and remove them:
rows_with_zero_avg = ba_beers_[ba_beers_['avg'] == 0]
#display(rows_with_zero_avg)
ba_beers_ = ba_beers_[ba_beers_['avg'] != 0]
ba_beers_ = ba_beers_.reset_index(drop=True)
# We are checking if there are  beers still left with 0 averages and remove them:
rows_with_zero_ratings=ba_beers_[ba_beers_['nbr_ratings'] == 0]
#display(rows_with_zero_ratings)
ba_beers_cleaned = ba_beers_[ba_beers_['nbr_ratings'] != 0]
ba_beers_cleaned = ba_beers_cleaned.reset_index(drop=True)
#ba_beers_cleaned is the dataset with both columns avg and nbr_ratings purified from the NaN and 0 values-
"""

"\n# We are checking if there are beers with 0 averages and remove them:\nrows_with_zero_avg = ba_beers_[ba_beers_['avg'] == 0]\n#display(rows_with_zero_avg)\nba_beers_ = ba_beers_[ba_beers_['avg'] != 0]\nba_beers_ = ba_beers_.reset_index(drop=True)\n# We are checking if there are  beers still left with 0 averages and remove them:\nrows_with_zero_ratings=ba_beers_[ba_beers_['nbr_ratings'] == 0]\n#display(rows_with_zero_ratings)\nba_beers_cleaned = ba_beers_[ba_beers_['nbr_ratings'] != 0]\nba_beers_cleaned = ba_beers_cleaned.reset_index(drop=True)\n#ba_beers_cleaned is the dataset with both columns avg and nbr_ratings purified from the NaN and 0 values-\n"

In [27]:
rb_beers_cleaned = rb_beers[~pd.isna(rb_beers['avg'])].reset_index() # avg = NaN valued beers are removed since there are not any reviews
rb_beers_cleaned

"""
# We are checking if there are beer with 0 ratings and remove them:
#rows_with_zero_ratings_rb=rb_beers_[rb_beers_['nbr_ratings'] == 0]
#display(rows_with_zero_ratings_rb)
rb_beers_ = rb_beers_[rb_beers_['nbr_ratings'] != 0].reset_index(drop=True)

# We are checking if there are beers still left with 0 averages and remove them:
#rows_with_zero_avg_rb = rb_beers_[rb_beers_['avg'] == 0]
#display(rows_with_zero_avg_rb)
rb_beers_cleaned = rb_beers_[rb_beers_['avg'] != 0].reset_index(drop=True)

display(rb_beers_cleaned)

#rb_beers_cleaned is the dataset with both columns avg and nbr_ratings purified from the NaN and 0 values-
"""

"\n# We are checking if there are beer with 0 ratings and remove them:\n#rows_with_zero_ratings_rb=rb_beers_[rb_beers_['nbr_ratings'] == 0]\n#display(rows_with_zero_ratings_rb)\nrb_beers_ = rb_beers_[rb_beers_['nbr_ratings'] != 0].reset_index(drop=True)\n\n# We are checking if there are beers still left with 0 averages and remove them:\n#rows_with_zero_avg_rb = rb_beers_[rb_beers_['avg'] == 0]\n#display(rows_with_zero_avg_rb)\nrb_beers_cleaned = rb_beers_[rb_beers_['avg'] != 0].reset_index(drop=True)\n\ndisplay(rb_beers_cleaned)\n\n#rb_beers_cleaned is the dataset with both columns avg and nbr_ratings purified from the NaN and 0 values-\n"

## Checking the correspondance between brewery_id in the beers DataFrames

In [28]:
rb_beers_cleaned[rb_beers_cleaned['brewery_id'] == 3198]

,index,beer_id,beer_name,brewery_id,brewery_name,style,nbr_ratings,avg
0,0,410549,33 Export (Gabon),3198,Sobraga,Pale Lager,1,2.72
1,1,105273,Castel Beer (Gabon),3198,Sobraga,Pale Lager,10,2.18
2,2,19445,Régab,3198,Sobraga,Pale Lager,27,1.83


In [29]:
rb_breweries[rb_breweries['id'] == 3198]

,id,location,name,nbr_beers
0,3198,Gabon,Sobraga,3


## Removing state information

In [30]:
import warnings
warnings.filterwarnings('ignore')

In [31]:
def edit_location(data_name):
    data_name_c = data_name.copy()
    for i in range(len(data_name['location'])):
        if len(data_name['location'][i]) > 10:
            if 'United States' in data_name['location'][i]: # Remove state names
                data_name_c['location'][i] = 'United States'
            elif ',' in data_name['location'][i]:
                data_name_c['location'][i] = data_name['location'][i][:(data_name['location'][i].index(','))] # Removing for the double names ( such as 'United Kingdom,England' )
            elif 'href' in data_name['location'][i]:
                data_name_c.drop(i)
    return data_name_c

In [32]:
ba_breweries_cleaned = edit_location(ba_breweries)
rb_breweries_cleaned = edit_location(rb_breweries)

In [33]:
ba_breweries_cleaned['location']

0           Kyrgyzstan
1           Kyrgyzstan
2           Kyrgyzstan
3           Kyrgyzstan
4           Kyrgyzstan
             ...      
16753          Germany
16754            Aruba
16755    United States
16756    United States
16757    United States
Name: location, Length: 16758, dtype: object

In [34]:
rb_breweries_cleaned['location'].value_counts()

location
United States            6921
England                  2124
Germany                  1999
Italy                    1051
Canada                    884
                         ... 
Sao Tome and Principe       1
Iraq                        1
Micronesia                  1
French Guiana               1
Gabon                       1
Name: count, Length: 217, dtype: int64

## Removing HTML links

In [35]:
# Done above

## Removing the countries who have too few reviewers

In [36]:
ba_users_cleaned = ba_users[~pd.isna(ba_users['location'])].reset_index() # location = NaN valued users are removed
rb_users_cleaned = rb_users[~pd.isna(rb_users['location'])].reset_index() # location = NaN valued users are removed
ba_users_cleaned

,index,nbr_ratings,nbr_reviews,user_id,user_name,location
0,0,7820,465,nmann08.184925,nmann08,"United States, Washington"
1,1,2521,2504,stjamesgate.163714,StJamesGate,"United States, New York"
2,2,1797,1143,mdagnew.19527,mdagnew,Northern Ireland
3,3,31,31,helloloser12345.10867,helloloser12345,Northern Ireland
4,4,604,604,cypressbob.3708,cypressbob,Northern Ireland
...,...,...,...,...,...,...
122420,153698,1,0,eturchick.374415,ETurchick,"United States, California"
122421,153699,1,1,everman.532342,Everman,"United States, California"
122422,153700,1,1,justin0001.352175,Justin0001,"United States, California"
122423,153702,1,1,joetex.800347,JoeTex,"United States, California"


In [37]:
rb_users_cleaned

,index,nbr_ratings,user_id,user_name,location
0,0,1890,175852,Manslow,Poland
1,1,89,442761,MAGICuenca91,Spain
2,2,169,288889,Sibarh,Poland
3,3,3371,250510,fombe89,Spain
4,4,13043,122778,kevnic2008,Germany
...,...,...,...,...,...
50587,70167,1,181614,HaraldS,Norway
50588,70169,1,134893,stamfordbus,England
50589,70170,1,327816,fobia405,Belgium
50590,70172,3,82020,klesidra,Slovenia


In [38]:
ba_users_cleaned_2 = edit_location(ba_users_cleaned)
rb_users_cleaned_2 = edit_location(rb_users_cleaned)

In [39]:
ba_users_cleaned_2

,index,nbr_ratings,nbr_reviews,user_id,user_name,location
0,0,7820,465,nmann08.184925,nmann08,United States
1,1,2521,2504,stjamesgate.163714,StJamesGate,United States
2,2,1797,1143,mdagnew.19527,mdagnew,Northern Ireland
3,3,31,31,helloloser12345.10867,helloloser12345,Northern Ireland
4,4,604,604,cypressbob.3708,cypressbob,Northern Ireland
...,...,...,...,...,...,...
122420,153698,1,0,eturchick.374415,ETurchick,United States
122421,153699,1,1,everman.532342,Everman,United States
122422,153700,1,1,justin0001.352175,Justin0001,United States
122423,153702,1,1,joetex.800347,JoeTex,United States


In [40]:
count = rb_users_cleaned_2['location'].value_counts() < 10 # We have to adjust the thresholds
count[count == True]

location
Andorra               True
Guatemala             True
Faroe Islands         True
Vietnam               True
Dominican Republic    True
                      ... 
Honduras              True
Falkland Islands      True
Kyrgyzstan            True
Papua New Guinea      True
Tibet                 True
Name: count, Length: 100, dtype: bool

In [41]:
rb_users_cleaned_2_copy = rb_users_cleaned_2.copy(deep=True)
for i in range(len(rb_users_cleaned_2['location'])):
    if rb_users_cleaned_2['location'][i] in count :
        rb_users_cleaned_2_copy.drop(i)

In [42]:
rb_users_cleaned_2_copy

,index,nbr_ratings,user_id,user_name,location
0,0,1890,175852,Manslow,Poland
1,1,89,442761,MAGICuenca91,Spain
2,2,169,288889,Sibarh,Poland
3,3,3371,250510,fombe89,Spain
4,4,13043,122778,kevnic2008,Germany
...,...,...,...,...,...
50587,70167,1,181614,HaraldS,Norway
50588,70169,1,134893,stamfordbus,England
50589,70170,1,327816,fobia405,Belgium
50590,70172,3,82020,klesidra,Slovenia


In [43]:
count2 = ba_users_cleaned_2['location'].value_counts() < 10 # We have to adjust the thresholds
count2[count2 == True]

location
Indonesia                                   True
Luxembourg                                  True
Antarctica                                  True
Uruguay                                     True
Bahamas                                     True
                                            ... 
South Georgia and South Sandwich Islands    True
Angola                                      True
Kazakhstan                                  True
Egypt                                       True
Sint Maarten                                True
Name: count, Length: 84, dtype: bool

In [44]:
ba_users_cleaned_2_copy = ba_users_cleaned_2.copy(deep=True)
for i in range(len(ba_users_cleaned_2['location'])):
    if ba_users_cleaned_2['location'][i] in count2 :
        ba_users_cleaned_2_copy.drop(i)

In [45]:
ba_users_cleaned_2_copy

,index,nbr_ratings,nbr_reviews,user_id,user_name,location
0,0,7820,465,nmann08.184925,nmann08,United States
1,1,2521,2504,stjamesgate.163714,StJamesGate,United States
2,2,1797,1143,mdagnew.19527,mdagnew,Northern Ireland
3,3,31,31,helloloser12345.10867,helloloser12345,Northern Ireland
4,4,604,604,cypressbob.3708,cypressbob,Northern Ireland
...,...,...,...,...,...,...
122420,153698,1,0,eturchick.374415,ETurchick,United States
122421,153699,1,1,everman.532342,Everman,United States
122422,153700,1,1,justin0001.352175,Justin0001,United States
122423,153702,1,1,joetex.800347,JoeTex,United States


# 1) Link between culture and taste

## a) Beer style preferences

In [46]:
# use clustering techniques to determine beer style is most popular in each country / geographic area
# use time information to determine if regional beer style preferences are stable (which would suggest that they are 
# strongly affected by culture)or if they vary over time

## b) Importance of specific beer attributes

In [47]:
# perform linear regression between attribute ratings the final rating for all countries together and compare coefficients for each attribute
# perform linear regression between attribute ratings the final rating for the different countries separately and observe the distribution of the coefficients for the different attributes across countries

# 2) Location-related biases in ratings

## a) Cultural biases

In [48]:
# determine the final rating for each country/ geographic area
# determine if the final rating for each country/ geographic area is the same using statistical tests

## b) Beer origin bias

In [49]:
# compare the final rating of domestic vs foreign beers and determine if there is a significant difference using statistical tests
# determine if the final rating of a given beer is correlated with the number of reviewers from the country where the beer comes from who reviewed that beer (scatter plot + Pearson’s correlation coefficient + regression)
# isolate beer enthusiasts (who wrote a very large number of reviews) and compare the final rating of domestic vs foreign beers and determine if there is a significant difference using statistical tests

# 3) Other biases

## a) Seasonal biases

In [50]:
# use the time information to determine the season during which each rating was posted (only consider countries with 4 seasons)
# group ratings by season
# within each group, determine the average final rating of each beer style
# compare the results for the different seasons

## b) Experience biais

In [51]:
# isolate users who gave a lot of ratings (based on a chosen threshold)
# for each user, sort their reviews chronologically and assign an "experience level" (predefined values that will be the same for all users: n<o<p) to each rating based the count of reviews posted by that user up to that rating: new reviewer (for the first n reviews), amateur (for the n+1 th review up to the oth review), expert (for the o+1 th review until the last review)
# calculate the average final rating for each experience level across all users
# represent results as a bar plot
# if a particular trend is visible,perform a paired t-test (for early vs. late reviews by the same user) to test if the rating decrease or increase is statistically significant